In [1]:
# File: custom_embedder.py (atau tambahkan ke utils.py)

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from typing import List
import numpy as np

class QwenEmbedder:
    """
    Kelas kustom untuk membuat embedding teks menggunakan model Qwen dari Hugging Face,
    mengimplementasikan proses secara manual tanpa wrapper LangChain.
    """
    def __init__(self, model_name: str, device: str = 'cpu'):
        """
        Menginisialisasi tokenizer dan model.
        
        Args:
            model_name (str): Nama model di Hugging Face Hub (cth: 'Qwen/Qwen3-Embedding-0.6B').
            device (str): Perangkat untuk menjalankan model ('cpu' atau 'cuda').
        """
        print(f"  > Memuat Tokenizer dan Model '{model_name}'...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        
        # Pindahkan model ke perangkat yang dipilih (CPU atau GPU)
        self.device = device
        self.model.to(self.device)
        self.model.eval() # Set model ke mode evaluasi (penting untuk inference)
        print("  > Model dan Tokenizer berhasil dimuat.")

    def _mean_pooling(self, model_output, attention_mask):
        """
        Melakukan Mean Pooling.
        Mengambil output dari model dan attention mask untuk menghasilkan satu vektor
        yang merepresentasikan seluruh kalimat.
        """
        # Langkah 1: Dapatkan semua embedding token dari output model
        token_embeddings = model_output.last_hidden_state
        
        # Langkah 2: Buat attention mask yang diperluas agar ukurannya cocok dengan token_embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        
        # Langkah 3: Kalikan embedding dengan mask untuk menolkan embedding dari token padding
        masked_embeddings = token_embeddings * input_mask_expanded
        
        # Langkah 4: Jumlahkan embedding yang relevan dan bagi dengan jumlah token yang relevan
        sum_embeddings = torch.sum(masked_embeddings, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
        return sum_embeddings / sum_mask

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        Membuat embedding untuk sekumpulan dokumen (teks).

        Args:
            texts (List[str]): Daftar string teks yang akan di-embed.

        Returns:
            List[List[float]]: Daftar dari embedding, di mana setiap embedding adalah daftar float.
        """
        if not texts or not isinstance(texts, list):
            return []
            
        # 1. TOKENISASI
        # Mengubah teks menjadi format yang bisa dibaca model (Token ID).
        # padding=True -> Menyamakan panjang semua kalimat dalam batch.
        # truncation=True -> Memotong kalimat yang terlalu panjang.
        # return_tensors='pt' -> Mengembalikan hasil sebagai PyTorch Tensors.
        encoded_input = self.tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            return_tensors='pt',
            max_length=1024 # Batas panjang token, bisa disesuaikan
        )
        
        # Pindahkan data token ke perangkat yang sama dengan model
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}

        # 2. INFERENCE MODEL
        # Matikan perhitungan gradien untuk mempercepat proses dan menghemat memori.
        with torch.no_grad():
            model_output = self.model(**encoded_input)

        # 3. POOLING
        # Mengubah output per-token menjadi satu vektor per-kalimat.
        sentence_embeddings = self._mean_pooling(model_output, encoded_input['attention_mask'])

        # 4. NORMALISASI
        # Menormalkan vektor agar panjangnya (magnitude) menjadi 1.
        # Ini penting untuk perhitungan cosine similarity.
        normalized_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        
        # Pindahkan hasil kembali ke CPU dan ubah menjadi list Python biasa
        return normalized_embeddings.cpu().numpy().tolist()

    def embed_query(self, query: str) -> List[float]:
        """
        Membuat embedding untuk satu teks (pertanyaan).

        Args:
            query (str): String pertanyaan.

        Returns:
            List[float]: Embedding untuk pertanyaan tersebut.
        """
        # Prosesnya sama, hanya saja inputnya adalah list dengan satu elemen.
        # [0] di akhir untuk mengambil embedding tunggal dari hasil list.
        return self.embed_documents([query])[0]


# --- CONTOH PENGGUNAAN ---
if __name__ == '__main__':
    # Inisialisasi embedder kita
    embedder = QwenEmbedder(model_name="Qwen/Qwen3-Embedding-0.6B")
    
    # Daftar kalimat untuk di-embed
    kalimat = [
        "Kasih itu sabar;",
        "Kasih itu murah hati;",
        "Ia tidak cemburu.",
        "Apa definisi dari kasih?"
    ]
    
    # Dapatkan embeddings
    embeddings = embedder.embed_documents(kalimat)
    
    # Tampilkan hasil embedding untuk kalimat pertama
    print("\nContoh Kalimat:", kalimat[0])
    print("Dimensi Embedding:", len(embeddings[0]))
    print("Contoh Embedding (5 elemen pertama):", np.array(embeddings[0][:5]))
    
    # Contoh embed satu query
    query_embedding = embedder.embed_query("Apa itu kasih?")
    print("\nContoh Query:", "Apa itu kasih?")
    print("Dimensi Embedding Query:", len(query_embedding))
    print("Contoh Embedding Query (5 elemen pertama):", np.array(query_embedding[:5]))

  > Memuat Tokenizer dan Model 'Qwen/Qwen3-Embedding-0.6B'...
  > Model dan Tokenizer berhasil dimuat.

Contoh Kalimat: Kasih itu sabar;
Dimensi Embedding: 1024
Contoh Embedding (5 elemen pertama): [-0.04304343  0.04892129 -0.012476    0.03305931  0.0044877 ]

Contoh Query: Apa itu kasih?
Dimensi Embedding Query: 1024
Contoh Embedding Query (5 elemen pertama): [-0.03473022 -0.05707059 -0.01321422  0.01108161  0.00415797]


In [2]:
class Mobil:
    def __init__(self, merk, warna):
        self.merk = merk
        self.warna = warna

    def jalan(self):
        print(f"Mobil {self.merk} berwarna {self.warna} sedang berjalan!")


In [4]:
mobil1 = Mobil("Toyota", "merah")
mobil2 = Mobil("Honda", "biru")

mobil1.jalan()
mobil2.jalan()


Mobil Toyota berwarna merah sedang berjalan!
Mobil Honda berwarna biru sedang berjalan!


In [ ]:
def tambah(a,b):
    hasil_tambah=a+b
    return hasil_tambah

def perkalian(c,d):
    hasil_perkalian=c*d
    return hasil_perkalian

In [6]:
pertambahan=tambah(5,7)
print(f"ini adakah hasil pertambahan {pertambahan}")

ini adakah hasil pertambahan 12


In [7]:
def perkalian(a,b):
    hasil_perkalian=a*b
    print("hasil_perkalian", hasil_perkalian)

In [8]:
perkalian(2,7)

hasil_perkalian 14


In [10]:
class matematika:
    def tambah(a,b):
        hasil_tambah=a+b
        return hasil_tambah

    def perkalian(c,d):
        hasil_perkalian=c*d
        return hasil_perkalian

In [ ]:
math=matematika
pertambahan=math.tambah(5,7)
print(pertambahan)

12


In [16]:
class matematikaself:
    def __init__(self,a,b):
        self.a=a
        self.b=b

    def pertambahan(self):
        hasil_tambah=self.a+self.b
        return hasil_tambah

In [18]:
math=matematikaself(5,7)
pertambahan=math.pertambahan()
print(pertambahan)

12


In [19]:
# File: custom_embedder.py (atau tambahkan ke utils.py)

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from typing import List
import numpy as np

class QwenEmbedder:
    def __init__(self, model_name: str, device: str = 'cpu'):
        print(f"  > Memuat Tokenizer dan Model '{model_name}'...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = device
        self.model.to(self.device)
        self.model.eval() 
        print("  > Model dan Tokenizer berhasil dimuat.")

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        masked_embeddings = token_embeddings * input_mask_expanded
        sum_embeddings = torch.sum(masked_embeddings, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        if not texts or not isinstance(texts, list):
            return []
        encoded_input = self.tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            return_tensors='pt',
            max_length=1024 
        )
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        sentence_embeddings = self._mean_pooling(model_output, encoded_input['attention_mask'])
        normalized_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return normalized_embeddings.cpu().numpy().tolist()

    def embed_query(self, query: str) -> List[float]:
        return self.embed_documents([query])[0]
if __name__ == '__main__':
    # embedder = QwenEmbedder(model_name="Qwen/Qwen3-Embedding-0.6B")
    
    # # Daftar kalimat untuk di-embed
    # kalimat = [
    #     "Kasih itu sabar;",
    #     "Kasih itu murah hati;",
    #     "Ia tidak cemburu.",
    #     "Apa definisi dari kasih?"
    # ]
    
    # # Dapatkan embeddings
    # embeddings = embedder.embed_documents(kalimat)
    
    # # Tampilkan hasil embedding untuk kalimat pertama
    # print("\nContoh Kalimat:", kalimat[0])
    # print("Dimensi Embedding:", len(embeddings[0]))
    # print("Contoh Embedding (5 elemen pertama):", np.array(embeddings[0][:5]))
    
    # # Contoh embed satu query
    # query_embedding = embedder.embed_query("Apa itu kasih?")
    # print("\nContoh Query:", "Apa itu kasih?")
    # print("Dimensi Embedding Query:", len(query_embedding))
    # print("Contoh Embedding Query (5 elemen pertama):", np.array(query_embedding[:5]))

SyntaxError: incomplete input (2605420543.py, line 68)